In [106]:
from sympy import *

In [107]:
divVs, divqD, Pt0, Pf0, phi0 = symbols('divVs, divqD, Pt0, Pf0, Φ0')
K_s, K_f, K_phi, eta_phi, sin_psi, gamma = symbols('Ks, Kf, KΦ, ηΦ, sinψ, λ̇')
Pt, Pf, dt = symbols('Pt, Pf, Δt')
phi = symbols('Φ_trial')

In [108]:
# K_f = K_phi
# K_s = K_phi


In [109]:
# divVs, divqD, Pt0, Pf0, phi0 = symbols('divVs, divqD, Pt0, Pf0, phi0')
# K_s, K_f, K_phi, eta_phi, sin_psi, gamma = symbols('K_s, K_f, K_phi, eta_phi, sin_psi, gamma')
# Pt, Pf, dt = symbols('Pt, Pf, dt')
# phi = symbols('phi')

In [110]:
elastic = 1
viscous = 1
plastic = 1
single_phase = 0

In [111]:
dPtdt  = (Pt-Pt0)/dt
dPfdt  = (Pf-Pf0)/dt
dphidt = elastic/K_phi*(dPfdt-dPtdt) + viscous/eta_phi*(Pf - Pt)

if single_phase==1:
    dphidt = 0
    phi = 0

dlnrhosdt = elastic * 1/(K_s*(1-phi)) * (dPtdt - phi*dPfdt) 
fpt       = dlnrhosdt - dphidt/(1-phi) + divVs
fpt       = fpt.simplify()

dlnrhofdt = dPfdt/K_f
fpf       = elastic * phi*dlnrhofdt + dphidt + phi*divVs + divqD
fpf       = fpf.simplify()

fphi = phi    - (phi0 + dphidt*dt)

display(fpt)
display(fpf)

(Ks*KΦ*divVs*Δt*ηΦ*(Φ_trial - 1) + Ks*(KΦ*Δt*(Pf - Pt) + ηΦ*(Pf - Pf0 - Pt + Pt0)) + KΦ*ηΦ*(-Pt + Pt0 + Φ_trial*(Pf - Pf0)))/(Ks*KΦ*Δt*ηΦ*(Φ_trial - 1))

(Kf*KΦ*Δt*ηΦ*(divVs*Φ_trial + divqD) + Kf*KΦ*Δt*(Pf - Pt) + Kf*ηΦ*(Pf - Pf0 - Pt + Pt0) + KΦ*Φ_trial*ηΦ*(Pf - Pf0))/(Kf*KΦ*Δt*ηΦ)

In [112]:
P_trial = solve([fpt, fpf], [Pt, Pf])

In [113]:
julia_code(P_trial[Pt].simplify())

'(-Kf .* Ks .* KΦ .* divVs .* Δt .^ 2 - Kf .* Ks .* KΦ .* divqD .* Δt .^ 2 - Kf .* Ks .* divVs .* Δt .* ηΦ - Kf .* Ks .* divqD .* Δt .* ηΦ - Kf .* KΦ .* Pf0 .* Δt .* Φ_trial + Kf .* KΦ .* Pt0 .* Δt - Kf .* KΦ .* divVs .* Δt .* Φ_trial .^ 2 .* ηΦ - Kf .* KΦ .* divqD .* Δt .* Φ_trial .* ηΦ - Kf .* Pt0 .* Φ_trial .* ηΦ + Kf .* Pt0 .* ηΦ + Ks .* KΦ .* Pf0 .* Δt .* Φ_trial + Ks .* KΦ .* divVs .* Δt .* Φ_trial .^ 2 .* ηΦ - Ks .* KΦ .* divVs .* Δt .* Φ_trial .* ηΦ + Ks .* Pt0 .* Φ_trial .* ηΦ + KΦ .* Pt0 .* Φ_trial .* ηΦ) ./ (-Kf .* KΦ .* Δt .* Φ_trial + Kf .* KΦ .* Δt - Kf .* Φ_trial .* ηΦ + Kf .* ηΦ + Ks .* KΦ .* Δt .* Φ_trial + Ks .* Φ_trial .* ηΦ + KΦ .* Φ_trial .* ηΦ)'

In [114]:
if single_phase==0:
    julia_code(P_trial[Pf].simplify())

In [115]:
if single_phase==0:
    julia_code(solve(fphi, phi)[0] .simplify())

In [116]:
# t, dt, phi0, Pt0, Pf0   = symbols('t, dt, phi0, Pt0, Pf0')
# phi1    = Function('phi')(t)
# Pt1     = Function('Pt')(t)
# Pf1     = Function('Pf')(t)
# Ps1     = (Pt1 - phi1*Pf1)/(1-phi1) 
# Ps1.subs(Pt1.diff(t), 'dPtdt').subs(Pf1.diff(t), 'dPfdt').diff(t).subs(Pf1, 'Pf').subs(Pt1, 'Pt').subs(phi1.diff(t), 'dphidt').subs(phi1, 'phi')

phi

Φ_trial

In [117]:
dPtdt  = (Pt-Pt0)/dt
dPfdt  = (Pf-Pf0)/dt
# phi    = phi0 + dphidt*dt

if single_phase==1:
    dphidt = plastic*gamma*sin_psi
else:
    # to be addressed (Thomsen)
    dphidt = elastic*(dPfdt-dPtdt)/K_phi + viscous*(Pf - Pt)/eta_phi + plastic*gamma*sin_psi
    
# dlnrhosdt = elastic * 1/K_s * (dPtdt - phi*dPfdt)/(1-phi) 
# Ps     = (Pt - phi*Pf)/(1-phi) 
dPsdt = (dPtdt - phi*dPfdt) /(1-phi)
# dPsdt = ((Pt - phi*Pf)/(1-phi) - (Pt0 - phi0*Pf0)/(1-phi0))/dt
# dPsdt = dphidt*(Pt - Pf*phi)/(1-phi)**2 + (dPtdt - phi*dPfdt - 0*Pf*dphidt) / (1 - phi)
dlnrhosdt = elastic * 1/K_s * ( dPsdt ) 

fpt = dlnrhosdt - dphidt/(1-phi) + divVs
fpt = fpt.simplify()

dlnrhofdt = dPfdt/K_f 
fpf  = elastic*phi*dlnrhofdt + dphidt + phi*divVs + divqD
fpf = fpf.simplify()

if single_phase==1:     
    P_corr = solve([fpt], [Pt])
else:
    P_corr = solve([fpt, fpf], [Pt, Pf])
P_corr

{Pt: (Kf*Ks*KΦ*divVs*Δt**2 + Kf*Ks*KΦ*divqD*Δt**2 + Kf*Ks*divVs*Δt*ηΦ + Kf*Ks*divqD*Δt*ηΦ + Kf*KΦ*Pf0*Δt*Φ_trial - Kf*KΦ*Pt0*Δt + Kf*KΦ*divVs*Δt*Φ_trial**2*ηΦ + Kf*KΦ*divqD*Δt*Φ_trial*ηΦ + Kf*KΦ*sinψ*Δt*Φ_trial*ηΦ*λ̇ + Kf*Pt0*Φ_trial*ηΦ - Kf*Pt0*ηΦ - Ks*KΦ*Pf0*Δt*Φ_trial - Ks*KΦ*divVs*Δt*Φ_trial**2*ηΦ + Ks*KΦ*divVs*Δt*Φ_trial*ηΦ - Ks*KΦ*sinψ*Δt*Φ_trial*ηΦ*λ̇ - Ks*Pt0*Φ_trial*ηΦ - KΦ*Pt0*Φ_trial*ηΦ)/(Kf*KΦ*Δt*Φ_trial - Kf*KΦ*Δt + Kf*Φ_trial*ηΦ - Kf*ηΦ - Ks*KΦ*Δt*Φ_trial - Ks*Φ_trial*ηΦ - KΦ*Φ_trial*ηΦ),
 Pf: (Kf*Ks*KΦ*divVs*Δt**2 + Kf*Ks*KΦ*divqD*Δt**2 + Kf*Ks*divVs*Δt*ηΦ + Kf*Ks*divqD*Δt*ηΦ + Kf*KΦ*Pf0*Δt*Φ_trial - Kf*KΦ*Pt0*Δt + Kf*KΦ*divVs*Δt*Φ_trial*ηΦ + Kf*KΦ*divqD*Δt*ηΦ + Kf*KΦ*sinψ*Δt*ηΦ*λ̇ + Kf*Pf0*Φ_trial*ηΦ - Kf*Pf0*ηΦ - Ks*KΦ*Pf0*Δt*Φ_trial - Ks*Pf0*Φ_trial*ηΦ - KΦ*Pf0*Φ_trial*ηΦ)/(Kf*KΦ*Δt*Φ_trial - Kf*KΦ*Δt + Kf*Φ_trial*ηΦ - Kf*ηΦ - Ks*KΦ*Δt*Φ_trial - Ks*Φ_trial*ηΦ - KΦ*Φ_trial*ηΦ)}

In [118]:
P_trial[Pt]

(Kf*Ks*KΦ*divVs*Δt**2 + Kf*Ks*KΦ*divqD*Δt**2 + Kf*Ks*divVs*Δt*ηΦ + Kf*Ks*divqD*Δt*ηΦ + Kf*KΦ*Pf0*Δt*Φ_trial - Kf*KΦ*Pt0*Δt + Kf*KΦ*divVs*Δt*Φ_trial**2*ηΦ + Kf*KΦ*divqD*Δt*Φ_trial*ηΦ + Kf*Pt0*Φ_trial*ηΦ - Kf*Pt0*ηΦ - Ks*KΦ*Pf0*Δt*Φ_trial - Ks*KΦ*divVs*Δt*Φ_trial**2*ηΦ + Ks*KΦ*divVs*Δt*Φ_trial*ηΦ - Ks*Pt0*Φ_trial*ηΦ - KΦ*Pt0*Φ_trial*ηΦ)/(Kf*KΦ*Δt*Φ_trial - Kf*KΦ*Δt + Kf*Φ_trial*ηΦ - Kf*ηΦ - Ks*KΦ*Δt*Φ_trial - Ks*Φ_trial*ηΦ - KΦ*Φ_trial*ηΦ)

In [119]:
dPt = ((P_corr[Pt] - P_trial[Pt])).simplify()
display(dPt)
print('ΔPt = ' +  julia_code(dPt))

KΦ*sinψ*Δt*Φ_trial*ηΦ*λ̇*(-Kf + Ks)/(-Kf*KΦ*Δt*Φ_trial + Kf*KΦ*Δt - Kf*Φ_trial*ηΦ + Kf*ηΦ + Ks*KΦ*Δt*Φ_trial + Ks*Φ_trial*ηΦ + KΦ*Φ_trial*ηΦ)

ΔPt = KΦ .* sinψ .* Δt .* Φ_trial .* ηΦ .* λ̇ .* (-Kf + Ks) ./ (-Kf .* KΦ .* Δt .* Φ_trial + Kf .* KΦ .* Δt - Kf .* Φ_trial .* ηΦ + Kf .* ηΦ + Ks .* KΦ .* Δt .* Φ_trial + Ks .* Φ_trial .* ηΦ + KΦ .* Φ_trial .* ηΦ)


In [120]:
if single_phase==0:
    dPf = ((P_corr[Pf] - P_trial[Pf])).simplify()
    julia_code(dPf)
    print('ΔPf = ' +  julia_code(dPf))
display(dPf)


ΔPf = Kf .* KΦ .* sinψ .* Δt .* ηΦ .* λ̇ ./ (Kf .* KΦ .* Δt .* Φ_trial - Kf .* KΦ .* Δt + Kf .* Φ_trial .* ηΦ - Kf .* ηΦ - Ks .* KΦ .* Δt .* Φ_trial - Ks .* Φ_trial .* ηΦ - KΦ .* Φ_trial .* ηΦ)


Kf*KΦ*sinψ*Δt*ηΦ*λ̇/(Kf*KΦ*Δt*Φ_trial - Kf*KΦ*Δt + Kf*Φ_trial*ηΦ - Kf*ηΦ - Ks*KΦ*Δt*Φ_trial - Ks*Φ_trial*ηΦ - KΦ*Φ_trial*ηΦ)